In [50]:
import os
import numpy as np
import pandas as pd
import pandas_ta as ta
from pathlib import Path
from datetime import datetime


candlestick_frame = 60*4
pnl_threshold = 3

downloads_path = str(Path.home() / "Downloads")
raw_data = f'{downloads_path}\\01-11-2019  15-08-2023.csv'
df = pd.read_csv(raw_data)
df['sma'] = ta.sma(df['close'], length=7)

ts = list(df['time'])
open = list(df['open'])
high = list(df['high'])
low = list(df['low'])
close = list(df['close'])
volume = list(df['volume'])
sma = list(df['sma'])


tot = len(ts)
sma_pc_lst = []
long_runup_lst = []
long_drawdown_lst = []
short_runup_lst = []
short_drawdown_lst = []

for idx in range(tot):
    sma_pc_calc = sma[idx]*100/sma[idx-1]
    sma_pc_lst.append(round(sma_pc_calc-100, 6)) if sma_pc_calc != None else sma_pc_lst.append(0)
    if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
        max_high = max(high[idx:idx+candlestick_frame])
        min_low = min(low[idx:idx+candlestick_frame])
        entry_price = open[idx]
        long_runup_lst.append(round((max_high*100/entry_price)-100, 6))
        long_drawdown_lst.append(round((min_low*100/entry_price)-100, 6))
        short_runup_lst.append(round((entry_price*100/min_low)-100, 6))
        short_drawdown_lst.append(round((entry_price*100/max_high)-100, 6))
    else:
        long_runup_lst.append(0)
        long_drawdown_lst.append(0)
        short_runup_lst.append(0)
        short_drawdown_lst.append(0)     

df['sma_pc'] = sma_pc_lst
df['long_runup'] = long_runup_lst
df['long_drawdown'] = long_drawdown_lst
df['short_runup'] = short_runup_lst
df['short_drawdown'] = short_drawdown_lst

exp_df_fn = f'{downloads_path}\\exp_df.csv'
df.to_csv(exp_df_fn, index=False)
# os.system(f"start EXCEL.EXE {exp_df_fn}")


In [51]:
import os
import numpy as np
import pandas as pd
import pandas_ta as ta
from pathlib import Path
from datetime import datetime



candlestick_frame = 60*4
pnl_threshold = 3
prev_sma_pc_count = 29


downloads_path = str(Path.home() / "Downloads")
processed_data_fn = f'{downloads_path}\\exp_df.csv'
processed_df = pd.read_csv(processed_data_fn)

ts = list(processed_df['time'])
sma_pc = list(processed_df['sma_pc'])
long_runup = list(processed_df['long_runup'])
short_runup = list(processed_df['short_runup'])

long=[]
short=[]
dont_trade=[]

tot = len(ts)

for idx in range(tot):
    if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
        app = {'idx': idx, 'ts': ts[idx]}
        for i in range(prev_sma_pc_count, -1, -1):
            app[f'sma_pc_{i}'] = sma_pc[idx-i]
        app['sma_pc_sum'] = sum(sma_pc[idx-prev_sma_pc_count: idx])
        if long_runup[idx] >= pnl_threshold:
            long.append(app)
        elif short_runup[idx] >= pnl_threshold:
            short.append(app)
        else:
            dont_trade.append(app)


long_fn = f'{downloads_path}\\long.csv'
pd.DataFrame(long).to_csv(long_fn, index=False)
short_fn = f'{downloads_path}\\short.csv'
pd.DataFrame(short).to_csv(short_fn, index=False)
dont_trade_fn = f'{downloads_path}\\dont_trade.csv'
pd.DataFrame(dont_trade).to_csv(dont_trade_fn, index=False)
